In [1]:
import requests
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from bs4 import BeautifulSoup
from analyses_code import preprocessing

import glob

## Preprocessing UK data

UK data has:
1. Count of border testing between the period of 2021-02-15 to 2021-03-05 from each country to the UK.
2. Variants of Concerns sequenced from border testing in that period
3. Individual vaccination status and their result

### Count of border testing

In [40]:
individual_testing = pd.read_csv(
    './dataset/UK_BT/2022-03-14_big_table.21_day_window.csv',
    usecols=[0, 1, 2, 3, 4])

In [41]:
# convert to necessary data format
individual_testing['first_test_date'] = pd.to_datetime(individual_testing['first_test_date'],format='%Y-%m-%d')
# individual_testing['first_test_date'] = pd.to_datetime(individual_testing['first_test_date'],format='%d/%m/%Y')

ordered_index = pd.date_range('2021-02','2022-02',freq='MS').strftime('%Y-%b')
individual_testing = individual_testing.drop(individual_testing[individual_testing['territory_name'] == '0'].index).reset_index(drop=True) # removed the ones with zero as territory name

Summarise this data into total tests count for each day

In [42]:
UK_border_testing_daily = pd.pivot_table(individual_testing,
                                         values=['positive','negative','total'],index=['first_test_date'],aggfunc=sum)
UK_border_testing_daily.to_csv('./dataset/preprocessed_data/UK_BT/UK_border_testing_daily.tsv')
UK_border_testing_daily

,negative,positive,total
first_test_date,,,
2021-02-15,287.0,4.0,291.0
2021-02-16,1336.0,45.0,1381.0
2021-02-17,3622.0,148.0,3770.0
2021-02-18,4297.0,148.0,4445.0
2021-02-19,5115.0,208.0,5323.0
...,...,...,...
2022-03-09,3688.0,119.0,3807.0
2022-03-10,3751.0,128.0,3879.0
2022-03-11,2136.0,101.0,2237.0


In [43]:
UK_border_testing_daily = pd.read_csv('./dataset/preprocessed_data/UK_BT/UK_border_testing_daily.tsv',index_col=[0])

Choose a date range, that is similar to the one available in Canada

In [4]:
UK_border_testing_daily_1Jul_25Dec = preprocessing.get_date_range(UK_border_testing_daily,'2021-07-01','2021-12-25')
# UK_border_testing_daily_1Jul_25Dec.to_csv('./dataset/preprocessed_data/UK_border_testing_daily_1Jul_25Dec.tsv')

In [22]:
UK_border_testing_daily_21Feb_4Jul = preprocessing.get_date_range(UK_border_testing_daily,'2021-02-21','2021-07-04')
UK_border_testing_daily_21Feb_4Jul.to_csv('./dataset/preprocessed_data/UK_BT/UK_border_testing_daily_21Feb_4Jul.tsv')

In [24]:
UK_border_testing_monthly_21Feb_4Jul = UK_border_testing_daily_21Feb_4Jul.groupby(
    by=[UK_border_testing_daily_21Feb_4Jul.index.year,UK_border_testing_daily_21Feb_4Jul.index.month]
)[['negative','positive','total']].sum()

In [114]:
UK_border_testing_monthly_21Feb_4Jul.to_csv('./dataset/preprocessed_data/UK_BT/UK_border_testing_monthly_21Feb_4Jul.tsv')

### Importing the VOC count

In [31]:
UK_VOC_daily = pd.read_csv('./dataset/UK_BT/2022-03-14_individ_sequences_byresultovertime.csv')
UK_VOC_daily['Earliest_Specimen_Date'] = pd.to_datetime(UK_VOC_daily['Earliest_Specimen_Date'])
UK_VOC_daily.to_csv('./dataset/preprocessed_data/UK_BT/UK_VOC_daily.tsv')
UK_VOC_daily['Variant'] = UK_VOC_daily['vam'] # change the name
UK_VOC_daily = UK_VOC_daily.drop(columns=['vam'])

In [32]:
#get the name of some of the known variants
VOC_uk = requests.get('https://www.gov.uk/government/publications/covid-19-variants-genomically-confirmed-case-numbers/variants-distribution-of-cases-data#Variant3').text
VOC_uk = BeautifulSoup(VOC_uk,'html.parser')
VOC_uk = pd.read_html(VOC_uk.find_all('table')[0].prettify())[0]
VOC_uk['Variant'] = [i.replace(' ','') for i in VOC_uk['Variant']]

In [33]:
UK_VOC_daily_23Feb_24Feb = preprocessing.get_date_range(UK_VOC_daily.set_index('Earliest_Specimen_Date'),
                            start = '2021-02-23',end='2022-02-24')
UK_VOC_daily_23Feb_24Feb = UK_VOC_daily_23Feb_24Feb.reset_index()

In [34]:
UK_VOC_daily_23Feb_24Feb.to_csv(
    './dataset/preprocessed_data/UK_BT/UK_VOC_border_testing_23Feb2021_24Feb2022.tsv',
    index=False)

In [35]:
individual_seq_Variants = UK_VOC_daily_23Feb_24Feb.pivot_table(
    values=['Count'], index=['Variant'], aggfunc=sum).reset_index()
individual_seq_Variants = individual_seq_Variants.merge(
    VOC_uk[['Variant', 'Lineage']], on='Variant', how='outer')

#VOC-21-NOV-01 is the Omicron B.1.1529
#VUI-21-OCT-01 is also known as AY.4.2
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VOC-21NOV-01','Lineage'] = 'B.1.1529'
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VOC-20DEC-01+E484K','Lineage'] = 'B.1.1.7 with E484K'
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VOC-21APR-02+K417N','Lineage'] = 'B.1.617.2 with K417N'
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VUI-21JUL-01','Lineage'] = 'B.1.621'
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VUI-21JUN-01','Lineage'] = 'C.37'
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VUI-21MAY-02','Lineage'] = 'C.36.3'
individual_seq_Variants.loc[individual_seq_Variants['Variant'] == 'VUI-21OCT-01','Lineage'] = 'AY.4.2'

In [36]:
individual_seq_Variants = individual_seq_Variants.set_index('Variant')

In [37]:
individual_seq_Variants = pd.DataFrame({
        'Variant':['Alpha VOC B.1.1.7','Beta VOC B.1.351','Delta VOC B.1.617.2','Gamma VOC P.1','Omicron B.1.1.529'],
        'Count':[individual_seq_Variants.loc[['VOC-20DEC-01',
                                'VOC-20DEC-01+E484K'],'Count'].sum(),
                individual_seq_Variants.loc['VOC-20DEC-02','Count'].sum(),
                individual_seq_Variants.loc[['VOC-21APR-02',
                                'VOC-21APR-02+E484K',
                                'VOC-21APR-02+K417N'],'Count'].sum(),
                individual_seq_Variants.loc['VOC-21JAN-02','Count'],
                individual_seq_Variants.loc['VOC-21NOV-01','Count']]
    })

In [38]:
individual_seq_Variants

,Variant,Count
0,Alpha VOC B.1.1.7,4754.0
1,Beta VOC B.1.351,274.0
2,Delta VOC B.1.617.2,23484.0
3,Gamma VOC P.1,66.0
4,Omicron B.1.1.529,33183.0


In [39]:
individual_seq_Variants.to_csv('./dataset/preprocessed_data/UK_BT/UK_VOC_border_testing_23Feb2021_24Feb2022.tsv',index=False)

### Vaccination status

In [11]:
UK_individual_vaxx = pd.read_csv('./dataset/UK_BT/2022-02-28_big_table.by_vaccination_status.csv')

In [17]:
UK_individual_vaxx.groupby(['first_test_date','vacc_status_clean']).sum()

positive  total
first_test_date vacc_status_clean                        
2021-02-15      full                             0      4
                no                               0     57
                part                             0      1
                vaccinated_unknown_doses         0     76
2021-02-16      full                             5     55
...                                            ...    ...
2022-02-26      full                            11     52
                no                               2    181
                part                             0      2
                vaccinated_unknown_doses         0     86
2022-02-27      full                             1      1

[1509 rows x 2 columns]

## Canada data

[Canada data](https://www.canada.ca/en/public-health/services/diseases/coronavirus-disease-covid-19/testing-screening-contact-tracing/summary-data-travellers.html#a3) has:
1. Border testing results from Feb 21 to July 4, 2021
2. Border testing results for partially/ unvaccinated travellers and vaccinated travellers
3. VOC counts between 23 Feb - Feb 24, 2022
4. Information about the travellers

In [2]:
html_content = requests.get(
    'https://www.canada.ca/en/public-health/services/diseases/coronavirus-disease-covid-19/testing-screening-contact-tracing/summary-data-travellers.html#a3'
).text

In [3]:
canada_soup = BeautifulSoup(html_content,'html.parser')
canada_tables = canada_soup.find_all('table')

### Border testing results from Feb 21 to July 4 2021

In [4]:
def Canada_border_testing(df):
    """
    df = dataFrame from Canada websit.
    with pd.MultiIndex in the columns (axis=1)
    and Air and Land and Overall pecent positive, will also calculate from the percentage the actual number
    
    """
    border_testing_positivity = df.copy()
    border_testing_positivity.columns = pd.MultiIndex.from_tuples([('Date','')]+border_testing_positivity.columns.to_list()[1:5]+[('Total','Percent positive')])
    border_testing_positivity[('Air','Tests completed')] = border_testing_positivity[('Air','Tests completed')].astype('int64')
    border_testing_positivity[('Land', 'Total tests completed')] = border_testing_positivity[('Land', 'Total tests completed')].astype('int64')
    
    border_testing_positivity[('Air','Percent positive')] = list(map(lambda x: float(x.replace('%','')),border_testing_positivity[('Air','Percent positive')]))
    border_testing_positivity[('Land','Percent positive')] = list(map(lambda x: float(x.replace('%','')),border_testing_positivity[('Land','Percent positive')]))
    border_testing_positivity[('Total', 'Percent positive')] = list(map(lambda x: float(x.replace('%','')),border_testing_positivity[('Total', 'Percent positive')]))
    
    border_testing_positivity[('Total','Total tests completed')] = border_testing_positivity[('Air','Tests completed')] + border_testing_positivity[('Land','Total tests completed')]
    
    border_testing_positivity[('Total','Test positive')] = border_testing_positivity[('Total', 'Percent positive')] * border_testing_positivity[('Total','Total tests completed')] / 100
    border_testing_positivity[('Total','Test positive')] = border_testing_positivity[('Total','Test positive')].round()
    
    border_testing_positivity = border_testing_positivity.set_index([('Date')])
    
    return border_testing_positivity

In [7]:
border_testing_positivity = pd.read_html(canada_tables[1].prettify())[0]
border_testing_positivity = border_testing_positivity.drop([6,7])

border_testing_positivity = Canada_border_testing(border_testing_positivity)
border_testing_positivity.index = [i.replace(u'\xa0',u'') for i in border_testing_positivity.index.tolist()]
border_testing_positivity.index.name = 'Date'
border_testing_positivity.to_csv('./dataset/preprocessed_data/Canada_BT/Canada_border_testing.tsv',sep='\t')

In [9]:
border_testing_positivity = pd.read_csv('./dataset/preprocessed_data/Canada_BT/Canada_border_testing.tsv',sep='\t', header=[0,1],index_col=[0],encoding='latin-1')

### VOC count Feb23- Feb24,2022

In [20]:
sequencing_result = pd.read_html(canada_tables[4].prettify())[0]

In [21]:
sequencing_result

,Mode,Variant of Concern (VOC) Total,Alpha VOC B.1.1.7,Beta VOC B.1.351,Delta VOC B.1.617.2,Gamma VOC P.1,Omicron B.1.1.529,Variants of Interest (VOI) Total
0,Border total,16993,2130,107,4318,149,10289,248
1,Land total,2916,370,19,721,45,1761,60
2,Air total,14077,1760,88,3597,104,8528,188


In [22]:
sequencing_result.to_csv('./dataset/preprocessed_data/Canada_BT/Canada_VOC_border_Testing.tsv')

### Traveller infomration (volume and citizenship)

In [22]:
border_testing_weekly_air_volumes_by_nationalities = pd.read_html(canada_tables[7].prettify())[0]

In [23]:
border_testing_cumulative_volumes = pd.read_html(canada_tables[5].prettify())[0]

In [25]:
border_testing_cumulative_volumes

,Mode,"Cumulative total March 21, 2020 – February 6, 2022","Total traveller volume January 1 - February 6, 2022",Total traveller volume 2021,Total traveller volume 2020,Total traveller volume 2019
0,Land,20324989,1223032,12335310,15906967,57380632
1,Air,9449965,1236651,6469230,9685737,36590236
2,Overall total travellers,29774954,2459683,18804540,25592704,93970868


In [263]:
border_testing_weekly_air_volumes = pd.read_html(canada_tables[6].prettify())[0]

In [269]:
border_testing_weekly_land_volumes = pd.read_html(canada_tables[8].prettify())[0]

In [26]:
border_testing_weekly_air_volumes_by_nationalities

Date  Footnote  1  \
                                    Date  Footnote  1   
0                       January 31 - February 6, 2022   
1                                 January 24-30, 2022   
2                                 January 17-23, 2022   
3                                 January 10-16, 2022   
4                                   January 3-9, 2022   
5                 December 27, 2021 – January 2, 2022   
6                                December 20-26, 2021   
7                                December 13-19, 2021   
8                                 December 6-12, 2021   
9                      November 29 – December 5, 2021   
10                               November 22-28, 2021   
11                               November 15-21, 2021   
12                                November 8-14, 2021   
13                                 November 1-7, 2021   
14                                October 25-31, 2021   
15                                October 18-24, 2021   
16                                October 11-17, 2021   
17                                 October 4-10, 2021   
18                     September 27 - October 3, 2021   
19                              September 20-26, 2021   
20                              September 13-19, 2021   
21                               September 6-12, 2021   
22                      August 30 - September 5, 2021   
23                                 August 23-29, 2021   
24                                 August 16-22, 2021   
25                                  August 9-15, 2021   
26                                   August 2-8, 2021   
27                           July 26 - August 1, 2021   
28                                   July 19-25, 2021   
29                                   July 12-18, 2021   
30                                    July 5-11, 2021   
31                               June 28-July 4, 2021   
32                                   June 21-27, 2021   
33                                   June 14-20, 2021   
34                                    June 7-13, 2021   
35                                May 31-June 6, 2021   
36                                    May 24-30, 2021   
37                                    May 17-23, 2021   
38                                    May 10-16, 2021   
39                                      May 3-9, 2021   
40  Footnotes  Footnote 1  Traveller volumes indic...   

                       Canadian citizens  Footnote  2  \
                                                 2022   
0                                              139481   
1                                              139143   
2                                              142544   
3                                              174415   
4                                              216752   
5                                                 NaN   
6                                                 NaN   
7                                                 NaN   
8                                                 NaN   
9                                                 NaN   
10                                                NaN   
11                                                NaN   
12                                                NaN   
13                                                NaN   
14                                                NaN   
15                                                NaN   
16                                                NaN   
17                                                NaN   
18                                                NaN   
19                                                NaN   
20                                                NaN   
21                                                NaN   
22                                                NaN   
23                                                NaN   
24                                                NaN   
25                                           

### Vaccination status Jul5,2021- Feb19,2022

In [125]:
Canada_testing_result_unvaccinated = pd.read_html(canada_tables[2].prettify())[0].drop(16)
Canada_testing_result_unvaccinated = Canada_border_testing(Canada_testing_result_unvaccinated)
Canada_testing_result_unvaccinated.to_csv('./dataset/preprocessed_data/Canada_BT/Canada_testing_result_unvaccinated.tsv',index=True)
# Canada_testing_result_unvaccinated = pd.read_csv('./dataset/preprocessed_data/Canada_BT/Canada_testing_result_unvaccinated.tsv',index_col=[0],header=[0,1])

In [126]:
Canada_testing_result_vaccinated = pd.read_html(canada_tables[3].prettify())[0].drop(16)
Canada_testing_result_vaccinated = Canada_border_testing(Canada_testing_result_vaccinated)
Canada_testing_result_vaccinated.to_csv('./dataset/preprocessed_data/Canada_BT/Canada_testing_result_vaccinated.tsv',index=True)
# Canada_testing_result_vaccinated = pd.read_csv('./dataset/preprocessed_data/Canada_BT/Canada_testing_result_vaccinated.tsv',index_col=[0],header=[0,1])

In [164]:
def get_total_summary(df):
    nov28_dec25 = pd.Series(df.iloc[3:7,:].sum(),name='Nov28-Dec25,2021')
    dec26_jan29 = pd.Series(df.iloc[7:12,:].sum(),name='Dec26-Jan29,2022')
    jan30_feb19 = pd.Series(df.iloc[12:15,:].sum(),name='Jan30-Feb19,2022')
    
    
    df = df.append([nov28_dec25, dec26_jan29, jan30_feb19])
    df = df.reset_index()
    df.columns = ['Date'] + df.columns.tolist()[1:]
    df = df.drop([3,4,5,6,7,8,9,10,11,12,13,14,15])
    df = df.set_index('Date')
    df = df.rename({'Total (July 5 to August 8, 2021)':'Jul5-Aug8,2021',
              'Total (August 9 to September 9, 2021)':'Aug9-Sep9,2021',
              'Total (September 10 to November 27, 2021)':'Sep10-Nov27,2021'})
    df['Percent positive'] = round(((df['Test positive']/ df['Total tests completed'])*100),2)
    return df

In [166]:
Canada_testing_result_vaccinated = get_total_summary(
    Canada_testing_result_vaccinated['Total'][[
        'Total tests completed', 'Test positive'
    ]])
Canada_testing_result_unvaccinated = get_total_summary(
    Canada_testing_result_unvaccinated['Total'][[
        'Total tests completed', 'Test positive'
    ]])

In [168]:
Canada_testing_result_vaccinated['Vaccinated'] = 'Vaccinated'
Canada_testing_result_unvaccinated['Vaccinated'] = 'Unvaccinated/Partially'

Canada_testing_result_vaxx = pd.concat([Canada_testing_result_unvaccinated,Canada_testing_result_vaccinated])

In [170]:
Canada_testing_result_vaxx.to_csv('./dataset/preprocessed_data/Canada_BT/Canada_testing_result_vaxx.csv',index=True)

## Iceland data

[Iceland](https://www.covid.is/data) has:
- Border Number of test from 15/06/2020 - 24/02/2022
- Percentage of positive tests
- 14-day incidence per 100 000 inhabitants?

In [24]:
from datetime import datetime

dateparse = lambda x: datetime.strptime(x, '%d.%m.%y')
Border_test_2020 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/Border_test_number/2020.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 2],
    names=['Date', 'Total tests'],
    header=0)
Border_test_2021 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/Border_test_number/2021.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 2],
    names=['Date', 'Total tests'],
    header=0)
Border_test_2022_1 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/Border_test_number/01.01.2022-10.02.2022.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 2],
    names=['Date', 'Total tests'],
    header=0)
Border_test_2022_2 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/Border_test_number/From 11.02.2022.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 3],
    names=['Date', 'Total tests'],
    header=0)

Iceland_total_tests = pd.concat([Border_test_2020,Border_test_2021,Border_test_2022_1,Border_test_2022_2])
Iceland_total_tests = Iceland_total_tests[~Iceland_total_tests.index.duplicated(keep='first')]

In [25]:
percentage_2020 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/positive_percentage/2020.csv',
    parse_dates=True,
    index_col=[0],
    date_parser=dateparse,
    usecols=[0, 2],
    names=['Date', 'percent positive'],
    header=0)
percentage_2021 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/positive_percentage/2021.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 2],
    names=['Date', 'percent positive'],
    header=0)
percentage_2022_1 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/positive_percentage/01.01.2022-10.02.2022.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 2],
    names=['Date', 'percent positive'],
    header=0)
percentage_2022_2 = pd.read_csv(
    './dataset/preprocessed_data/Iceland_BT/raw/positive_percentage/From 11.02.2022.csv',
    parse_dates=True,
    date_parser=dateparse,
    index_col=[0],
    usecols=[0, 3],
    names=['Date', 'percent positive'],
    header=0)
Iceland_percent_positive = pd.concat([percentage_2020,percentage_2021,percentage_2022_1,percentage_2022_2])
# Iceland_percent_positive = Iceland_percent_positive[~Iceland_percent_positive.index.duplicated(keep='first')]

In [26]:
Iceland_total_tests = pd.concat([Iceland_total_tests,Iceland_percent_positive],axis=1)

Iceland_total_tests = Iceland_total_tests[~Iceland_total_tests['Total tests'].isna()]


In [27]:
percent_positive = [float(i.replace(',','.').replace('%','')) if type(i)==str else np.nan for i in Iceland_total_tests['percent positive']]
Iceland_total_tests['percent positive'] = percent_positive

In [28]:
Iceland_total_tests['positive'] = np.ceil((Iceland_total_tests['percent positive'] * Iceland_total_tests['Total tests'])/ 100)
Iceland_total_tests = Iceland_total_tests.drop('percent positive',axis=1)

In [29]:
Iceland_total_tests.to_csv('./dataset/preprocessed_data/Iceland_BT/Iceland_total_test.csv',index=True)

## Israel data

[Israel](https://datadashboard.health.gov.il/COVID-19/general?utm_source=go.gov.il&utm_medium=referral) has:
- Border testing daily divided by country
- vaccine status

In [2]:
border_testing = pd.read_excel('./dataset/preprocessed_data/Israel_BT/raw/border_testing_Israel.xlsx',header=[1])

border_testing.columns = [
    'Date', 'Hebrew_Country', 'Positive residents', 'Positive foreigners',
    'Positive resident percentage', 'Positive foreigners percentage'
]

border_testing['Date'] = pd.to_datetime(border_testing['Date'],
                                        format='%d-%m-%Y')

In [3]:
unique_countries = border_testing['Hebrew_Country'].unique()
translated_countries = [
    'Guinea', 'Lesotho', 'Latvia', 'Montenegro', 'Mexico', 'South Korea',
    'France', 'United States', 'Turkey', 'Singapore', 'Sri Lanka (photo)',
    'Egypt', 'Moldova', 'Cyprus', 'Kazakhstan', 'Italy', 'Lithuania',
    'Bosnia and Herzegovina', 'Belgium', 'Denmark', 'Ecuador', 'Guatemala',
    'Georgia', 'Sao Tome and Principe', 'Maldives', 'Russia', 'Sweden',
    'Botswana', 'Greece', 'Romania', 'Belarus', 'Serbia', 'Dominican Harp',
    'Armenia', 'Brazil', 'Zimbabwe', 'Finland', 'Ivory Coast', 'Angola',
    'Australia', 'Japan', 'Canada', 'Austria', 'Philippines', 'Czech Republic',
    'Ukraine', 'South Sudan', 'Azerbaijan', 'Malta', 'Puerto Rico', 'Nigeria',
    'Tanzania', 'Kenya', 'New Zealand', 'Hong Kong', 'Ethiopia', 'Norway',
    'Colombia', 'Ireland', 'United Kingdom', 'Poland', 'Uganda', 'Morocco',
    'Nepal', 'Hungary', 'Rwanda', 'Ghana', 'Luxembourg', 'Zambia', 'Thailand',
    'Costa Rica', 'UAE', 'Slovenia', 'Spain', 'Germany', 'Zaire',
    'Netherlands', 'China', 'Portugal', 'Croatia', 'Turkmenistan',
    'South Africa', 'Switzerland', 'All Countries', 'Chile', 'Seychelles',
    'Iceland', 'Peru', 'Indonesia', 'Taiwan', 'Uzbekistan', 'Tunisia',
    'Venezuela', 'Jordan', 'Iraq', 'Saudi Arabia', 'Kyrgyzstan', 'Bulgaria',
    'Sierra Leone', 'Uruguay', 'Albania', 'Syria', 'Bolivia', 'Paraguay',
    'Panama', 'India', 'Cuba', 'Senegal', 'Argentina', 'Burundi', 'Oman',
    'Lebanon', 'Mali', 'Mozambique', 'Netherlands Antilles',
    'Wallis and Futuna Islands', 'Estonia', 'Cameroon', 'Madagascar', 'Qatar',
    'Gabon', 'Bahrain', 'Namibia', 'Malawi', 'Central African Rap', 'Vietnam',
    'El Salvador', 'Monaco', 'Honduras', 'San Marino', 'Macedonia',
    'Tajikistan', 'Antigua and Barbuda', 'Virgin Islands', 'Bahamas',
    'Liberia', 'Barbados', 'Kosovo', 'Togo', 'Congo', 'Laos', 'Kiribati',
    'St Vincent', 'Reunion', 'Grande', 'French Polynesia', 'New Guinea',
    'Bermuda', 'French Guiana', 'New Caledonia', 'Mauritius', 'Gamaica',
    'Gibraltar', 'Cape Verde', ' Equatorial Guinea ', ' Libya ', ' Swaziland ',
    ' Tsad ', 'Gambia', 'Turks and Caicos Islands', 'Burkina Faso', 'Dominica',
    'Cambodia', 'Malaysia', 'Vanuatu', 'Samoa', 'England', 'Kuwait',
    'Mongolia', 'Algeria', 'Haiti', 'Piggy Islands',
    'Gilbert and Alice Islands', 'Slovakia', 'Martinique', 'Tuvalu', 'Andorra',
    'Eritrea', 'Pakistan', 'Nicaragua', 'Niger', 'Palau', 'Sudan',
    'Marshall Islands', 'Belize', 'Afghanistan', 'North Korea', 'Suriname',
    'Brunei', 'Papua', 'Myanmar (Burma)', 'Cook Islands', 'Yemen',
    'Benin (Dahumi)', 'Tonga', 'Macau', 'Iran', 'Bangladesh', 'Guam',
    'East Timor', 'Somalia', 'Trinidad and Tobago', 'Giboti',
    'British Solomon Islands', 'Norfolk Islands', 'Falkland Islands', 'Guyana',
    'St. Lucia ', ' Liechtenstein ', ' Congo, Ref. Deep. ', ' Peanut ',
    'Nauru', 'Comoros', 'Guadeloupe', 'Mauritania', 'Guinea-Bissau',
    'Cayman Islands', ' St. Keats and Nevis'
]

country_dict = dict(zip(unique_countries, translated_countries))
border_testing['Country'] = [
    country_dict[country] for country in border_testing['Hebrew_Country']
]

In [4]:
border_testing['Total residents'] = border_testing['Positive residents'] / (border_testing['Positive resident percentage']/
                                                                          100)
border_testing['Total foreigners'] = border_testing['Positive foreigners'] / (border_testing['Positive foreigners percentage']/
                                                                          100)
border_testing = border_testing.replace([np.inf, -np.inf], np.nan)
border_testing = border_testing.fillna(0)

In [86]:
vaccination_status = pd.read_excel('./dataset/preprocessed_data/Israel_BT/raw/vaccine_status.xlsx',header=[1])
vaccination_status.columns = [
    'Date', 'Hebrew_Country', 'Positive and vaccinated',
    'Positive and vaccinated (not valid)', 'Positive unvaccinated',
    'Vaccinated residents', 'Vaccinated(not valid) residents',
    'Unvaccinated residents'
]
vaccination_status['Date'] = pd.to_datetime(vaccination_status['Date'],format='%d-%m-%Y')
vaccination_status['Country'] = [country_dict[country] for country in vaccination_status['Hebrew_Country']]

In [88]:
vaccination_status.to_csv('./dataset/preprocessed_data/Israel_BT/vaccination_status.csv',index=False)

In [6]:
border_testing['Total positive'] = border_testing['Positive residents'] + border_testing['Positive foreigners']
border_testing['Total tests'] = border_testing['Total residents'] + border_testing['Total foreigners']
border_testing_total = border_testing[['Date','Total positive','Total tests','Country']].copy()
# border_testing_total = border_testing_total[border_testing_total['Country']=='All Countries']
border_testing_total['Total tests'] = np.ceil(border_testing_total['Total tests'])

In [9]:
border_testing_total.to_csv('./dataset/preprocessed_data/Israel_BT/border_testing_total.csv',index=False)

In [5]:
border_testing_total = pd.read_csv('./dataset/preprocessed_data/Israel_BT/border_testing.csv')

## Korea data

Korea data can be scrapped from the Wayback machine

In [ ]:
from analyses_code import wayback_webscrapping
import time
import pandas as pd
import tqdm

url = 'http://ncov.mohw.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun='
wbm = wayback_webscrapping.Waybackmachine(url)
wbm.get_date_range('2020-04-26','2022-03-15')

for date in tqdm.tqdm(wbm.all_dates):
    website_html = wbm.get_html(date)
    time.sleep(2)
    table = wbm.look_for_table(website_html,caption_text='해외유입 환자현황')
    if not table:
        continue
    table = wbm.save_table(table,date)
    table.to_csv(f'./dataset/preprocessed_data/Korea_BT/raw/{date}.txt',index=False)

In [41]:
Korea_border_testing = pd.concat([pd.read_csv(file) for file in glob.glob('./dataset/preprocessed_data/Korea_BT/raw/*.txt')])

In [27]:
Korea_border_testing.to_csv('./dataset/preprocessed_data/Korea_BT/raw/Korea_border_testing.raw',index=False)

In [42]:
Korea_border_testing.columns = ['Korean_Type','Korean_Location','New Cases','Total Cases','Ratio','Date']

In [43]:
korean_dict = dict(
    zip(Korea_border_testing['Korean_Location'].unique(), [
        'Total', 'China', 'Asia ex-China', 'Europe', 'America', 'Africa',
        'Australia', 'Quarantine stage', 'Local', 'Korean National',
        'Foreigner', 'America', 'Oceania'
    ]))
korean_dict.update(
    zip(Korea_border_testing['Korean_Type'].unique(),
        ['Total', 'Imported Country', 'Confirmation Stage', 'Nationality']))

In [44]:
Korea_border_testing['Location'] = [korean_dict[term] for term in Korea_border_testing['Korean_Location']]
Korea_border_testing['Type'] = [korean_dict[term] for term in Korea_border_testing['Korean_Type']]
Korea_border_testing['Date'] = pd.to_datetime(Korea_border_testing['Date'],format='%Y%m%d')

In [55]:
Korea_border_testing_new_cases_daily = Korea_border_testing.loc[
    (Korea_border_testing['Type'] == 'Imported Country'),
    ['Date', 'Location', 'New Cases']]

In [56]:
Korea_border_testing_new_cases_daily.to_csv(
    './dataset/preprocessed_data/Korea_BT/Korea_border_testing_new_cases_daily.csv',
    index=False)

In [57]:
Korea_border_testing_new_cases_daily

,Date,Location,New Cases
1,2020-04-21,China,0
2,2020-04-21,Asia ex-China,1
3,2020-04-21,Europe,1
4,2020-04-21,America,3
5,2020-04-21,Africa,0
...,...,...,...
2,2022-03-15,Asia ex-China,69
3,2022-03-15,Europe,13
4,2022-03-15,America,23
5,2022-03-15,Africa,3


## Thailand data

In [71]:
thai_data = requests.get('https://covid19.ddc.moph.go.th/api/Cases/timeline-cases-all')
thai_data = thai_data.json()
thai_data = pd.DataFrame(thai_data)

In [74]:
thai_data.columns

Index(['txn_date', 'new_case', 'total_case', 'new_case_excludeabroad',
       'total_case_excludeabroad', 'new_death', 'total_death', 'new_recovered',
       'total_recovered', 'update_date'],
      dtype='object')

In [75]:
thai_data['new_cases_abroad'] = thai_data['new_case'] - thai_data['new_case_excludeabroad']
thai_data['total_cases_abroad'] = thai_data['total_case'] - thai_data['total_case_excludeabroad']

In [80]:
thai_data_border_testing = thai_data[['txn_date','new_cases_abroad']]
thai_data_border_testing.columns = ['Date','Positive tests']

In [81]:
thai_data_border_testing.to_csv('./dataset/preprocessed_data/Thai_BT/thai_data_border_testing.csv',index=False)